# Final project - FWI 
## Team by:
### Emilio Ochoa
### Naomi Guevara
### Liumei Ma

Hello! I am putting the MATLAB code in parallel with the python code. Feel free to optimize the best as you can to obtain the final .py file

``load('RecordedData.mat', 'x', 'y', 'C', 'x_circ', 'y_circ', 'f', 'REC_DATA');``

In [1]:
from scipy.io import loadmat
import jax
import jax.numpy as jnp
from fwi_loss_function import fwi_loss_function
from solve_helmholtz import solve_helmholtz


data = loadmat('RecordedData1.mat')
x = data['x'].squeeze()
y = data['y'].squeeze()
C = data['C']
x_circ = data['x_circ'].squeeze()
y_circ = data['y_circ'].squeeze()
f = data['f'].item()
REC_DATA = data['REC_DATA']


``dwnsmp = 1;
tx_include = 1:dwnsmp:numElements;
REC_DATA = REC_DATA(tx_include,:);`` 

In [2]:
dwnsmp = 1
num_elements = x_circ.size
a0 = 10
L_PML = 9.0e-3
tx_include = jnp.arange(0, num_elements, dwnsmp)
REC_DATA = REC_DATA[tx_include, :]


``elemInclude = true(numElements, numElements);
for tx_element = 1:numElements 
    elemLeftRightExclCurrent = elemLeftRightExcl + tx_element;
    elemLeftRightExclCurrent(elemLeftRightExclCurrent<1) = numElements + ...
         elemLeftRightExclCurrent(elemLeftRightExclCurrent<1);
    elemLeftRightExclCurrent(elemLeftRightExclCurrent>numElements) = ...
        elemLeftRightExclCurrent(elemLeftRightExclCurrent>numElements) - numElements;
    elemInclude(tx_element,elemLeftRightExclCurrent) = false;
end
``

In [3]:
numElemLeftRightExcl = 31
elemLeftRightExcl = jnp.arange(-numElemLeftRightExcl, numElemLeftRightExcl + 1)
elemInclude = jnp.ones((num_elements, num_elements), dtype=bool)

for tx_element in range(num_elements):
    elemLeftRightExclCurrent = (elemLeftRightExcl + tx_element) % num_elements
    elemInclude = elemInclude.at[tx_element, elemLeftRightExclCurrent].set(False)


``dxi = 0.8e-3; xmax = 120e-3;
xi = -xmax:dxi:xmax; yi = xi;
[Xi, Yi] = meshgrid(xi, yi);
``

In [4]:
dxi = 0.8e-3
xmax = 120e-3
xi = jnp.arange(-xmax, xmax + dxi, dxi)
yi = xi
Xi, Yi = jnp.meshgrid(xi, yi, indexing='xy')


``x_idx = dsearchn(xi(:), x_circ(:));
y_idx = dsearchn(yi(:), y_circ(:));
ind = sub2ind([Nyi, Nxi], y_idx, x_idx);
``

This part maybe can be optimized?

In [5]:
from scipy.spatial import cKDTree

xi_tree = cKDTree(xi[:, None])
x_idx = xi_tree.query(x_circ[:, None])[1]

yi_tree = cKDTree(yi[:, None])
y_idx = yi_tree.query(y_circ[:, None])[1]

Nyi, Nxi = yi.size, xi.size
ind = y_idx * Nxi + x_idx


In [6]:
SRC = jnp.zeros((Nyi, Nxi, tx_include.size))
for i, tx_elmt_idx in enumerate(tx_include):
    x_idx_src = x_idx[tx_elmt_idx]
    y_idx_src = y_idx[tx_elmt_idx]
    SRC = SRC.at[y_idx_src, x_idx_src, i].set(1)

print('SRC shape:', SRC.shape)


SRC shape: (301, 301, 256)


In [7]:
from jaxopt import LBFGS

# Initial guess for slowness
c_init = 1480.0  # m/s
SLOW_ESTIM = jnp.ones((Nyi, Nxi)) / c_init

# Wrap loss function to pass extra arguments
objective = lambda slow: fwi_loss_function(slow, xi, yi, SRC, f, a0, L_PML, REC_DATA, ind, elemInclude, tx_include, num_elements)

# Initialize and run LBFGS solver
solver = LBFGS(fun=objective, maxiter=1)
result = solver.run(SLOW_ESTIM)
SLOW_ESTIM = result.params



/Users/emilio/anaconda3/envs/jax_env/lib/python3.12/site-packages/jax/_src/numpy/lax_numpy.py:5548: ComplexWarning: Casting complex values to real discards the imaginary part
  out_array: Array = lax_internal._convert_element_type(
/Users/emilio/anaconda3/envs/jax_env/lib/python3.12/site-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=complex64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/Users/emilio/anaconda3/envs/jax_env/lib/python3.12/site-packages/jax/_src/ops/scatter.py:137: ComplexWarning: Casting complex values to real discards the imaginary part
  return lax_internal._convert_element_type(out, dtype, weak_type)


TypeError: grad requires real-valued outputs (output dtype that is a sub-dtype of np.floating), but got complex64. For holomorphic differentiation, pass holomorphic=True. For differentiation of non-holomorphic functions involving complex outputs, use jax.vjp directly.

In [ ]:
import matplotlib.pyplot as plt

VEL_ESTIM = 1.0 / SLOW_ESTIM

plt.subplot(1, 2, 1)
plt.imshow(C, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='gray')
plt.title("True Velocity")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(VEL_ESTIM, extent=[xi.min(), xi.max(), yi.min(), yi.max()], origin='lower', cmap='gray')
plt.title("Estimated Velocity")
plt.colorbar()

plt.tight_layout()
plt.show()


: 

: 